<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Mean-Variance Portfolio Theory

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Theory

Topics of interest include:

* diversification effects
* possible risk-return combinations
* efficient frontier

## Real World Data

**Instead of artificial `NumPy` or `pandas` data, real-world end-of-day time series data for financial assets is used.**

## Financial Assets

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
np.set_printoptions(suppress=True)

In [ ]:
url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

### Returns

In [ ]:
symbols = ['AAPL.O', 'MSFT.O']

In [ ]:
data = raw[symbols]

In [ ]:
rets1 = np.log(data / data.shift(1))

In [ ]:
rets1.head()

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'GLD']# , 'EUR=', '.VIX']

In [ ]:
data = raw[symbols]

In [ ]:
rets2 = np.log(data / data.shift(1))

In [ ]:
rets2.head()

## Portfolio Return

In [ ]:
rets = rets2

In [ ]:
def port_return(phi):
    return np.dot(rets.mean() * 252, phi)

In [ ]:
noa = rets.shape[1]
noa

In [ ]:
phi = noa * [1 / noa]
phi

In [ ]:
port_return(phi)

## Portfolio Risk

In [ ]:
def port_risk(phi):
    return math.sqrt(np.dot(phi, np.dot(rets.cov() * 252, phi)))

In [ ]:
port_risk(phi)

## Mean-Variance Portfolios

In [ ]:
rets = rets1
noa = rets1.shape[1]
noa

In [ ]:
p = np.random.random((500, noa))
p = (p.T / p.sum(axis=1)).T
p.sum(axis=1)[:5]

In [ ]:
rr1 = np.array([(port_risk(phi), port_return(phi)) for phi in p])

In [ ]:
rets = rets2
noa = rets2.shape[1]
noa

In [ ]:
p = np.random.random((5000, noa))
p = (p.T / p.sum(axis=1)).T
p.sum(axis=1)[:5]

In [ ]:
%time rr2 = np.array([(port_risk(phi), port_return(phi)) for phi in p])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr1[:, 0], rr1[:, 1], 'ro')
plt.plot(rr2[:, 0], rr2[:, 1], 'bo')
plt.xlabel('risk')
plt.ylabel('return');

## Efficient Frontier

In [ ]:
from scipy.optimize import minimize

### Fixing the Target Return

In [ ]:
noa

In [ ]:
phi

In [ ]:
bnds = noa * [(0, 1)]

In [ ]:
cons = [{'type': 'eq', 'fun': lambda phi: phi.sum() - 1},
       {'type': 'eq', 'fun': lambda phi: port_return(phi) - target}]

In [ ]:
ret = np.linspace(0.09, 0.225, 25)

In [ ]:
ef = list()
for target in ret:
    opt = minimize(port_risk, phi, bounds=bnds, constraints=cons)
    ef.append((opt['fun'], target))
ef = np.array(ef)

In [ ]:
ef[:5]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr1[:, 0], rr1[:, 1], 'ro', alpha=0.3)
plt.plot(rr2[:, 0], rr2[:, 1], 'bo', alpha=0.3)
plt.plot(ef[:, 0], ef[:, 1], 'y^', ms=8)
plt.xlabel('risk')
plt.ylabel('return');

### Fixing the Target Volatility

In [ ]:
cons = [{'type': 'eq', 'fun': lambda phi: phi.sum() - 1},
       {'type': 'eq', 'fun': lambda phi: port_risk(phi) - target}]

In [ ]:
vol = np.linspace(0.1225, 0.25, 25)

In [ ]:
ef = list()
for target in vol:
    opt = minimize(lambda phi: -port_return(phi), phi,
                   bounds=bnds, constraints=cons)
    ef.append((target, -opt['fun']))
ef = np.array(ef)

In [ ]:
ef[:5]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr1[:, 0], rr1[:, 1], 'ro', alpha=0.3)
plt.plot(rr2[:, 0], rr2[:, 1], 'bo', alpha=0.3)
plt.plot(ef[:, 0], ef[:, 1], 'y^', ms=8)
plt.xlabel('risk')
plt.ylabel('return');

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>